In [1]:
from google.cloud import storage
from google.oauth2 import service_account
from more_itertools import bucket
from pandera import Column, Check
from schema import Schema, And, Use, Optional, SchemaError
import json
import os
import io
import pandas as pd
import pandera as pa

with open('..\security\parameters.json', 'r') as json_file:
    parameters = json.load(json_file)

bucket_name = parameters['bucket_name']

credentials = service_account.Credentials.from_service_account_file("..\security\parameters-key.json")
storage_client = storage.Client(credentials=credentials)
bucket = storage_client.bucket(bucket_name)

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from schema import Schema, And, Use

schema_hired_employees1 = Schema({
    'id': And(Use(int)),
    'name': And(Use(str)),
    'datetime': And(Use(str)),
    'de_id': And(Use(int)),
    'job_id': And(Use(int))
})

schema_departments1 = Schema({
    'id': And(Use(int)),
    'department': And(Use(str))
})

schema_jobs1 = Schema({
    'id': And(Use(int)),
    'job': And(Use(str))
})


In [31]:


columns_hired_employees = ['id', 'name', 'datetime', 'departament_id', 'job_id']
columns_jobs = ['id', 'job']
columns_departments = ['id', 'department']


In [37]:
def validate_dataframe(filename, columns_names, schema_table, chunksize = 1000):
    #Variables 
    data_validate = pd.DataFrame(columns=columns_names)
    data_error = pd.DataFrame(columns=columns_names)
    indexs_errors = pd.Series([],dtype=pd.Int32Dtype()) 

    #Data Blob
    blob = bucket.blob(filename, chunk_size = 256 * 1024)
    data = blob.download_as_string()
   
    for chunk in pd.read_csv(io.BytesIO(data), names=columns_names, header=None, chunksize=chunksize):
        try:
            schema_table(chunk, lazy=True)
        except pa.errors.SchemaErrors as exc:
            indexs_errors = pd.concat([indexs_errors, exc.failure_cases["index"]])
            print('Cantidad de errores: ', len(indexs_errors.index))
        data_validate = pd.concat([data_validate, chunk[~chunk.index.isin(indexs_errors)]])
        data_error = pd.concat([data_error, chunk[chunk.index.isin(indexs_errors)]])

    return data_validate, data_error

In [39]:
blobs = storage_client.list_blobs(bucket_name)

for blob in blobs:
    if 'poc-ingest/' in blob.name  and '.csv' in blob.name and 'logs' not in  blob.name:
        filename = blob.name
        name = filename.replace('poc-ingest/','').replace('.csv','')
        columns_name = globals()['columns_' + name]
        schema_table = globals()['schema_' + name]
        ok, error = validate_dataframe(filename, columns_name, schema_table)
        print(error)

Empty DataFrame
Columns: [id, department]
Index: []
Cantidad de errores:  35
Cantidad de errores:  74
        id                   name              datetime  departament_id  \
1        2               Ty Hofer  2021-05-30T05:43:46Z             8.0   
66      67           Thia Morican  2021-03-10T19:27:10Z             NaN   
83      84         Ludvig Norwood  2021-02-26T18:47:53Z             3.0   
86      87  Cirstoforo Martinetto  2021-10-15T09:19:20Z             NaN   
96      97          Beltran Natte  2021-11-01T05:12:01Z             NaN   
...    ...                    ...                   ...             ...   
1857  1858                    NaN  2021-06-28T03:01:01Z             6.0   
1859  1860          Marin Slipper  2022-02-08T10:39:02Z             4.0   
1868  1869         Cesaro Solomon                   NaN             6.0   
1924  1925             Cary Tothe                   NaN             3.0   
1986  1987         Shaina Wyldish  2021-12-13T10:05:09Z             NaN  

In [5]:
blob2 = bucket.blob('poc-ingest/jobs.csv', chunk_size = 256 * 1024)
data2 = blob2.download_as_string()
#print(data)

In [6]:
chunksize = 1000
chunk_validate = pd.DataFrame(columns=columns_jobs)
chunk_error = pd.DataFrame(columns=columns_jobs)
for chunk in pd.read_csv(io.BytesIO(data2), names=columns_jobs, header=None, chunksize=chunksize):

    try:
        schema_jobs(chunk, lazy=True)
    except pa.errors.SchemaErrors as exc:
        print(exc.failure_cases["index"])

        #chunk_validate.append(chunk[~chunk.index.isin(exc.failure_cases["index"])])
        #chunk_error.append(chunk[chunk.index.isin(exc.failure_cases["index"])])

print(chunk_error)

Empty DataFrame
Columns: [id, job]
Index: []


In [12]:
blob = bucket.blob('poc-ingest/hired_employees.csv', chunk_size = 256 * 1024)
data = blob.download_as_string()
#print(data)

chunksize = 1000
data_validate = pd.DataFrame(columns=columns_hired_employees)
data_error = pd.DataFrame(columns=columns_hired_employees)
indexs_errors = pd.Series([],dtype=pd.Int32Dtype()) 
for chunk in pd.read_csv(io.BytesIO(data), names=columns_hired_employees, header=None, chunksize=chunksize):
    try:
        schema_hired_employees(chunk, lazy=True)
    except pa.errors.SchemaErrors as exc:
        indexs_errors = pd.concat([indexs_errors, exc.failure_cases["index"]])
        print('Cantidad de errores: ', len(indexs_errors.index))
    
    data_validate = pd.concat([data_validate, chunk[~chunk.index.isin(indexs_errors)]])
    data_error = pd.concat([data_error, chunk[chunk.index.isin(indexs_errors)]])



In [18]:
blob = bucket.blob('poc-ingest/logs/data_logs.csv')

blob.upload_from_string(
            data=data_error.to_csv(index=False),
            content_type='text/csv'
        )


In [9]:
chunksize = 1000
data_validate = []
data_error = []
for chunk in pd.read_csv(io.BytesIO(data), names=columns_test, header=None, chunksize=chunksize):
    for index, row in chunk.itertuples(): 
        try:
            schema_test.validate(row.to_dict())
            data_validate.append(row)
        except SchemaError as error:
            data_error.append(row)


NameError: name 'columns_test' is not defined

In [ ]:
print(data_validate)

[id                             1
name                 Harold Vogt
datetime    2021-11-07T02:48:42Z
de_id                        2.0
job_id                      96.0
Name: 0, dtype: object, id                             3
name                 Lyman Hadye
datetime    2021-09-01T23:27:38Z
de_id                        5.0
job_id                      52.0
Name: 2, dtype: object, id                             4
name               Lotti Crowthe
datetime    2021-10-01T13:04:21Z
de_id                       12.0
job_id                      71.0
Name: 3, dtype: object, id                             5
name              Gretna Lording
datetime    2021-10-10T22:22:17Z
de_id                        6.0
job_id                      80.0
Name: 4, dtype: object, id                             6
name              Marlow Antecki
datetime    2021-04-23T23:45:42Z
de_id                        6.0
job_id                      95.0
Name: 5, dtype: object, id                             7
name                J

In [ ]:
def check(schema_table, data):
    try:
        schema_table.validate(data)
        return True
    except SchemaError:
        return False


In [ ]:
print(check(conf_schema, conf))

In [ ]:
filename = "..\\files_csv\\hired_employees.csv"

chunksize = 1000
for chunk in pd.read_csv(filename, chunksize=chunksize):
    #print(chunk.count)
    print(chunk.size)


5000
4990
